# load libraries

In [2]:
import xarray as xr
import dask
from dask.distributed import Client, LocalCluster

# set data file

In [7]:
data_file = "test_file.nc"

# start workers

In [4]:
client = Client(n_workers=2, threads_per_worker=1,
                memory_limit='350MB', dashboard_address=':8787')

In [5]:
client

Client Scheduler: tcp://127.0.0.1:44699 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 700.00 MB


In [ ]:
# client.close()

# open data

In [28]:
# open and have a look
## ds = xr.open_dataset(data_file)
## ds

In [8]:
ds = xr.open_dataset(data_file, chunks = {"xt_ocean": 112, "yt_ocean": 242})
ds

<xarray.Dataset>
Dimensions:          (nv: 2, st_ocean: 152, time: 12, xt_ocean: 224, yt_ocean: 242)
Coordinates:
  * xt_ocean         (xt_ocean) float64 8.283 8.383 8.483 ... 30.38 30.48 30.58
  * yt_ocean         (yt_ocean) float64 53.86 53.91 53.96 ... 65.81 65.86 65.91
  * time             (time) object 2012-01-16 12:00:00 ... 2012-12-16 12:00:00
  * st_ocean         (st_ocean) float64 0.25 0.7508 1.257 ... 263.0 265.0 267.0
Dimensions without coordinates: nv
Data variables:
    st_ocean_bounds  (st_ocean, nv) float64 dask.array<chunksize=(152, 2), meta=np.ndarray>
    t_zoo            (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(12, 152, 242, 112), meta=np.ndarray>
    time_bounds      (time, nv) object dask.array<chunksize=(12, 2), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.7
    institution:     Leibniz Institute for Baltic Sea Research Warnemuende, P...
    title:           Biogeochemical and basic physical state variables calcul...
    acknowl

In [10]:
ds.t_zoo.data

dask.array<open_dataset-e21f5f3e7e803e36a0459057b061871ct_zoo, shape=(12, 152, 242, 224), dtype=float32, chunksize=(12, 152, 242, 112)>

In [17]:
t_zoo = ds.t_zoo

In [29]:
# client.scatter(t_zoo, workers = ['localhost:39661', 'localhost:34049'])

# processing data

In [26]:
zoo_mean_delay = t_zoo.mean(dim = 'time')
zoo_mean_delay

<xarray.DataArray 't_zoo' (st_ocean: 152, yt_ocean: 242, xt_ocean: 224)>
dask.array<mean_agg-aggregate, shape=(152, 242, 224), dtype=float32, chunksize=(152, 242, 112)>
Coordinates:
  * xt_ocean  (xt_ocean) float64 8.283 8.383 8.483 8.583 ... 30.38 30.48 30.58
  * yt_ocean  (yt_ocean) float64 53.86 53.91 53.96 54.01 ... 65.81 65.86 65.91
  * st_ocean  (st_ocean) float64 0.25 0.7508 1.257 1.771 ... 263.0 265.0 267.0

In [27]:
zoo_mean_comp = zoo_mean_delay.compute(workers = ['localhost:39661', 'localhost:34049'])

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker

KeyboardInterrupt

distributed.nanny - WARNING - Restarting worker
